# Train Model

## Load base model

In [ ]:
%cd ..

In [ ]:
from fastai.vision.all import *

In [ ]:
def label_func(x):
    return str(x['file']).replace("images", "masks/obs0_rep0").replace("image","mask")

In [ ]:
def acc_seg(input, target):
    target = target.squeeze(1)
    return (input.argmax(dim=1)==target).float().mean()

def multi_dice(input:Tensor, targs:Tensor, class_id=0, inverse=False):
    n = targs.shape[0]
    input = input.argmax(dim=1).view(n,-1)
    # replace all with class_id with 1 all else with 0 to have binary case
    output = (input == class_id).float()
    # same for targs
    targs = (targs.view(n,-1) == class_id).float()
    if inverse:
        output = 1 - output
        targs = 1 - targs
    intersect = (output * targs).sum(dim=1).float()
    union = (output+targs).sum(dim=1).float()
    res = 2. * intersect / union
    res[torch.isnan(res)] = 1
    return res.mean()

def diceComb(input:Tensor, targs:Tensor):
    return multi_dice(input, targs, class_id=0, inverse=True)
def diceLV(input:Tensor, targs:Tensor):
    return multi_dice(input, targs, class_id=1)
def diceMY(input:Tensor, targs:Tensor):
    return multi_dice(input, targs, class_id=2)

In [ ]:
trainedModel = load_learner("inputs/models/kaggle-ukbb-base-fastai2.pkl", cpu=False)

## Prepare data loading

In [ ]:
all_files = sorted(glob.glob("inputs/cmr-cine-sscrofa/data/png/images/*.png"))

In [ ]:
df = pd.DataFrame({"file":all_files})
df

In [ ]:
df = df.assign(
    id=lambda x: [z.split("/")[-1].split("_")[0] for z in x['file']],
    frame=lambda x: [int(z.split("/")[-1].split("_")[2][5:8]) for z in x['file']],
)
df

In [ ]:
sets = pd.read_csv("inputs/training/cmr-cine-sscrofa.sets.tsv", sep="\t")

In [ ]:
df = pd.merge(df, sets)

In [ ]:
esed = pd.read_csv("inputs/cmr-cine-sscrofa/data/metadata/obs0_rep0.tsv", sep="\t")
esed = esed.melt(id_vars=['id'], value_vars=['es', 'ed'], var_name="phase", value_name="frame")
esed

In [ ]:
df = pd.merge(df, esed)

In [ ]:
df = df.assign(is_valid=lambda x: x['set']=="val")

In [ ]:
df.set.value_counts()

In [ ]:
train_val = df[df.set != "test"]

In [ ]:
train_val.set.value_counts()

In [ ]:
heart = DataBlock(blocks=(ImageBlock, MaskBlock(codes = np.array(["background","left_ventricle","myocardium"]))),
        get_x=ColReader("file"),
        get_y = label_func,
        splitter = ColSplitter(col="is_valid"),
        item_tfms=Resize(512, method='crop'),
        batch_tfms=aug_transforms(do_flip=True,max_rotate=90,max_lighting=.4,max_zoom=1.2,size=256))

In [ ]:
import ctypes

magma_path = '/tank/home/ankenbrand/miniconda3/lib/libmagma.so'
magma_path = '/tank/home/ankenbrand/miniconda3/pkgs/magma-2.5.4-h6103c52_2/lib/libmagma.so'
libmagma = ctypes.cdll.LoadLibrary(magma_path)
libmagma.magma_init()

In [ ]:
dls = heart.dataloaders(train_val, bs=8)

In [ ]:
dls.show_batch()

## Retrain model (with 50+50 epochs)

In [ ]:
trainedModel.dls = dls

In [ ]:
trainedModel.path = Path("model")

In [ ]:
trainedModel.add_cbs([CSVLogger(fname="50_50.csv",append=True)])

In [ ]:
trainedModel.freeze()

In [ ]:
trainedModel.lr_find()

In [ ]:
for i in range(5):
    trainedModel.fit_one_cycle(10, lr_max=1e-4)
    trainedModel.save("50_50-{}-epochs".format(10*i+10))

In [ ]:
trainedModel.show_results()

In [ ]:
trainedModel.unfreeze()

In [ ]:
trainedModel.lr_find()

In [ ]:
for i in range(5):
    trainedModel.fit_one_cycle(10, lr_max=1e-5)
    trainedModel.save("50_50-{}-epochs-unfrozen".format(10*i+10))

In [ ]:
trainedModel.remove_cb(CSVLogger)
trainedModel.export("50_50-final.pkl")

## Retrain model with DiceLoss (and 50+50 epochs)

In [ ]:
trainedModel = load_learner("inputs/models/kaggle-ukbb-base-fastai2.pkl", cpu=False)

In [ ]:
trainedModel.dls = dls

In [ ]:
trainedModel.loss_func = DiceLoss()

In [ ]:
trainedModel.path = Path("model")

In [ ]:
trainedModel.add_cbs([CSVLogger(fname="dice_50_50.csv",append=True)])

In [ ]:
trainedModel.freeze()

In [ ]:
trainedModel.lr_find()

In [ ]:
for i in range(5):
    trainedModel.fit_one_cycle(10, lr_max=1e-4)
    trainedModel.save("dice_50_50-{}-epochs".format(10*i+10))

In [ ]:
trainedModel.show_results()

In [ ]:
trainedModel.unfreeze()

In [ ]:
trainedModel.lr_find()

In [ ]:
for i in range(5):
    trainedModel.fit_one_cycle(10, lr_max=1e-5)
    trainedModel.save("dice_50_50-{}-epochs-unfrozen".format(10*i+10))

In [ ]:
trainedModel.remove_cb(CSVLogger)
trainedModel.export("dice_50_50-final.pkl")